# Spectral fitting and model selection with pycroscopy

### Rama Vasudevan

### Px Credits: Gerd Duscher, Suhas Somnath, S. Mani Valleti, Maxim Ziatdinov, Rama Vasudevan

Let's look at how we can fit some spectra with pycroscopy, and then perfor model selection

In [ ]:
%matplotlib notebook
!pip install -U pyNSID sidpy SciFiReaders nanonispy gwyfile pycroscopy
#!pip install pip install threadpoolctl==3.1.0

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

import pyNSID
import sidpy as sid
import SciFiReaders as sr
from sidpy.proc.fitter import SidFitter

# Load and Visualize Spectral dataset

Here we load a dataset of I-V curves captured by conductive atomic force microscopy, on a BiFeO3 sample.
The data has been transformed so that we plot not the log of the current density J, as a function of the square root of the electric field. I-V curves plotted in this fashion will be linear if they follow Schottky emission form. The dataset was originally presented in the paper <a href = "https://www.nature.com/articles/s41467-017-01334-5"> Vasudevan et al. Nat Commun. 8, 1318 (2017) </a>

Here we will load the file, which was saved as a pyNSID HDF5 file, using SciFiReaders, which reads it into a <i>sidpy.Dataset</i> object

Then we will use the .plot() method of the <i>sidpy.Dataset</i> object for interactive visualization.


In [ ]:
data_path = r'data/bfo_iv_final.hf5'
dr = sr.NSIDReader(data_path)
dataset_sid = dr.read()[0]
dataset_sid.plot();


# Functional Fitting

Much of the data appears linear. We can define any function of our choice and use sidpy's <i>SidFitter</i> class to make it easy to perform the fit on all of the spectra. We simply define the fit function, instantiate the class, and then call the do_fit() method.

Advantages of using the fitter class include:
- Innate scalability: we leverage the parallelism of Dask, so that the computations are performed in parallel
- Superior priors: We can use a k-means cluster approach to improve priors for the function fits, as described in <a href = "https://iopscience.iop.org/article/10.1088/2632-2153/abfbba/meta">Creange et al.</a>
- Dimension handling: There is intelligent folding and unfolding within the class to handle simple cases. For more complex situations (e.g., when there are multiple spectral dimensions, but only one is used for the fitting), these can be specified too.


In [ ]:
#Define the function we want each spectrum to

def one_lin_func(xvec, *coeff):
    a1,a2 = coeff
    return a1*xvec + a2

#Instantiate the SidFitter class
fitter = SidFitter(dataset_sid, one_lin_func,num_workers=4,
                           threads=2, return_cov=False, return_fit=True, return_std=False,
                           km_guess=True,num_fit_parms = 2)

In [ ]:
fit_parameters, fitted_dataset = fitter.do_fit() #With one line we can fit all the spectra

## Visualize Fit Results

We can visualize the parameter maps from the fititng, or visualize the fitted data curves

### Visualize coeffient maps

The functional fits provided us with coefficient maps for the fit parameters.

We can simply call the .plot() method again to visualize them.

In [ ]:
fit_parameters.plot();

### Visualize fitted spectra
Each spectrum is associated with a specific fit. 

We can visualize them again easily
through calling the <i>visualize_fit_results()</i> method of the SidFitter class.

In [ ]:
fitter.visualize_fit_results() # We can visualize the results of the fit interactively too...

# Exercises

1. Create your own dataset, with a different fitting function (model), and use the sidpy tools to make it a sidpy dataset object (see here for an example). Then fit the data to the model. Compare the results you get, for when you set the km_guess=True to km_guess = False for your dataset. Does it make much of a difference?


2. (Advanced): what if there is more than one possible model that we can fit to the data? How would we test which model is 'better' to use? (Hint: this <a href = "https://nirpyresearch.com/akaike-information-criterion-for-model-selection/"> link </a> is handy). Try to create a dataset where the spatial response varies and is described by one model in some pixels and another at other pixels, and see how the AIC scores compare
